## Импорт и установка библиотек

In [1]:
#!pip install dbfread # file reader
#!pip install patool # rar unarchiver

In [1]:
import pandas as pd
import patoolib
import os
import numpy as np
import shutil

from dbfread import DBF

In [2]:
def read_dbf(file):
    return pd.DataFrame(iter(DBF(file, encoding='cp866')))

## Разархивирование файлов

### Чтение файлов

- Файл mmyyyy**_B.DBF**, где mmyyyy – отчетный месяц, отчетный год: Данные формы 0409101 по кредитным организациям, раскрывающим свою информацию в Интернет (в соответствии с Указанием Банка России от 27.03.1998 No192-У утратило силу в 2009 году)
- Файл mmyyyy**B1.DBF**, где mmyyyy – отчетный месяц, отчетный год: Данные формы 0409101 по кредитным организациям, раскрывающим свою информацию в Интернет (в соответствии с письмом Банка России от 21.12.2006 No165-Т – в полном объеме и с оговоркой)
- Файл mmyyyy**_N.DBF**, где mmyyyy – отчетный месяц, отчетный год: Перечень кредитных организаций, раскрывающих данные формы 0409101 в Интернет (в
соответствии с Указанием Банка России от 27.03.1998 No192-У утратило силу в 2009 году)
- Файл mmyyyy**N1.DBF**, где mmyyyy – отчетный месяц, отчетный год: Перечень кредитных организаций, раскрывающих данные формы 0409101 в Интернет (в соответствии с письмом Банка России от 21.12.2006 No165-Т – в полном объеме и с оговоркой)
- Файл mmyyyy**_S.DBF**, где mmyyyy – отчетный месяц, отчетный год: Контрольная сумма для проверки корректности передачи информации в соответствии с Указанием Банка России от 27.03.1998 No192-У (утратило силу в 2009 году) (для проверки корректности передачи файла «mmyyyy_B.DBF»):
- Файл mmyyyy**S1.DBF**, где mmyyyy – отчетный месяц, отчетный год: Контрольные суммы для проверки корректности передачи информации в соответствии с письмом Банка России от 21.12.2006 No165-Т (для проверки корректности передачи файла «mmyyyyB1.DBF»)
- Файл NAMES.DBF: План счетов бухгалтерского учета в кредитных организациях


In [28]:
df = pd.DataFrame()

In [29]:
for filename in os.listdir('data/rars'):
    if not filename.startswith('.'):
        name = filename.split('.')[0]

        patoolib.extract_archive(f'data/rars/{filename}', outdir=f'data/{name}', verbosity=-1)
        folder_contents = os.listdir(f'data/{name}')
        filename = [fname for fname in folder_contents if fname.startswith('PK')][0]
        frame = read_dbf(f'data/{name}/{filename}')
        #names = read_dbf(f'data/{name}/F802META.DBF')
        #frame = frame.merge(names[['NUM_SC', 'NAME']], on='NUM_SC')[chosen_columns]
        date = name.split('-')[1]
        frame = frame.rename(columns={
            'KORR_P': 'KORR_P_'+date,
            'KORR_M': 'KORR_M_'+date,
            'VSEGO': 'VSEGO'+date
        })
        
        df = pd.concat([df, frame], join='outer', axis=1)

        shutil.rmtree(f'data/{name}')

df = df.reset_index(drop=True)

INFO patool: ... creating output directory `data/802-20190701'.
INFO patool: ... creating output directory `data/802-20180101'.
INFO patool: ... creating output directory `data/802-20190101'.
INFO patool: ... creating output directory `data/802-20180701'.
INFO patool: ... creating output directory `data/802-20190401'.
INFO patool: ... creating output directory `data/802-20201001'.
INFO patool: ... creating output directory `data/802-20171001'.
INFO patool: ... creating output directory `data/802-20180401'.
INFO patool: ... creating output directory `data/802-20211001'.
INFO patool: ... creating output directory `data/802-20200101'.
INFO patool: ... creating output directory `data/802-20210701'.
INFO patool: ... creating output directory `data/802-20200701'.
INFO patool: ... creating output directory `data/802-20210101'.
INFO patool: ... creating output directory `data/802-20181001'.
INFO patool: ... creating output directory `data/802-20210401'.
INFO patool: ... creating output directo

In [30]:
df.to_csv('data/df.csv')

## Обработка показателей

Рассматриваемые показатели:

1) Отношение собственных средств к чистым активам
2) Отношение резервов к чистым активам
3) Чистые активы или их логарифм
4) Отношение просроченной задолженности по ссудам к кредитам экономики
5) Отношение просроченной задолженности по ссудам к обязательным резервам в ЦБ РФ
6) Отношение оборотов по корреспондентским счетам за период времени к чистым активам
7) Отношение балансовой прибыли к чистым активам
8) Отношение чистой прибыли к чистым активам
9) Норматив текущей ликвидности
10) Отношение ликвидных активов к чистым активам
11) Отношение депозитов физических лиц до 30 дней к депозитам физических лиц
12) Отношение объема негосударственных ценных бумаг к чистым активам
